In [22]:
import pandas as pd

senteval_setfit = pd.read_csv("setfit/setfit_senteval_results.csv")
senteval_setfit["ID"] = ["setfit"] * len(senteval_setfit)
senteval_finetuned = pd.read_csv("finetuned/finetuned_senteval_results.csv")
senteval_finetuned["ID"] = ["finetuned"] * len(senteval_finetuned)
merged = pd.concat([senteval_setfit, senteval_finetuned])
model_col = merged.model.tolist()
dataset_col = ["sst2" if "sst2" in model else "sarcastic" for model in model_col]
model_types = merged.model.apply(lambda x: "-".join(x.replace("USERNAME/", "").split("-")[:2]))
model_types = model_types.apply(lambda x: "minilm-6" if x == "all-MiniLM" else x)
merged["model_type"] = model_types
merged["dataset"] = dataset_col
merged = merged.drop("model", axis=1)
merged = merged.drop(columns=["SICKEntailment", "MRPC"])
merged.loc[merged.ID == "pretrained", "dataset"] = "-"
merged

,MR,CR,SUBJ,MPQA,SST2,TREC,avg,ID,model_type,dataset
0,84.30,88.85,90.91,86.08,89.18,86.0,84.27,setfit,gte-base,sst2
1,81.61,86.52,90.01,87.50,88.69,86.0,81.92,setfit,gte-base,sarcastic
2,85.43,85.16,86.58,83.93,91.05,88.0,82.18,setfit,e5-small,sst2
3,82.69,83.97,90.65,86.80,88.80,90.2,81.62,setfit,e5-small,sarcastic
0,71.20,66.44,86.57,79.63,80.94,74.4,74.61,finetuned,minilm-6,sarcastic
1,81.21,84.53,87.43,84.76,86.49,81.2,81.53,finetuned,minilm-6,sst2
2,82.40,76.27,90.47,85.75,89.95,71.4,78.81,finetuned,e5-small,sarcastic
3,88.95,88.98,91.06,86.28,93.41,79.8,84.97,finetuned,e5-small,sst2
4,84.33,88.82,92.82,88.04,90.83,88.4,85.01,finetuned,gte-base,sarcastic
5,89.31,89.27,92.91,85.95,93.19,80.8,85.50,finetuned,gte-base,sst2


In [23]:
# sort by AVG score
merged = merged.sort_values("avg", ascending=False)
merged = merged.reset_index(drop=True)

merged = merged.rename(columns={"ID": "Variant", "model_type": "Base model", "dataset": "Dataset"})
cols = merged.columns.tolist()
cols = cols[-3:] + cols[:-3]
merged = merged[cols]
merged = merged.reset_index(drop=True)

print(merged.to_latex(float_format="%.2f", index=False))

\begin{tabular}{lllrrrrrrr}
\toprule
Variant & Base model & Dataset & MR & CR & SUBJ & MPQA & SST2 & TREC & avg \\
\midrule
finetuned & gte-base & sst2 & 89.31 & 89.27 & 92.91 & 85.95 & 93.19 & 80.80 & 85.50 \\
finetuned & gte-base & sarcastic & 84.33 & 88.82 & 92.82 & 88.04 & 90.83 & 88.40 & 85.01 \\
finetuned & e5-small & sst2 & 88.95 & 88.98 & 91.06 & 86.28 & 93.41 & 79.80 & 84.97 \\
finetuned & gte-small & sst2 & 87.72 & 89.59 & 90.85 & 86.86 & 91.38 & 79.00 & 84.83 \\
setfit & gte-base & sst2 & 84.30 & 88.85 & 90.91 & 86.08 & 89.18 & 86.00 & 84.27 \\
setfit & e5-small & sst2 & 85.43 & 85.16 & 86.58 & 83.93 & 91.05 & 88.00 & 82.18 \\
setfit & gte-base & sarcastic & 81.61 & 86.52 & 90.01 & 87.50 & 88.69 & 86.00 & 81.92 \\
finetuned & gte-small & sarcastic & 80.51 & 83.52 & 90.17 & 86.11 & 87.59 & 84.60 & 81.84 \\
setfit & e5-small & sarcastic & 82.69 & 83.97 & 90.65 & 86.80 & 88.80 & 90.20 & 81.62 \\
finetuned & minilm-6 & sst2 & 81.21 & 84.53 & 87.43 & 84.76 & 86.49 & 81.20 & 81.53